# BERT Training with Loss-Based Batch Selection

This notebook demonstrates training BERT on IMDB with two strategies:
1. **Normal**: Standard training on all examples
2. **Loss-based**: Filter examples based on previous loss

**Note:** For quick testing in Colab, we'll use a subset of data. Remove the subset limits for full training.

## 1. Install Required Packages

In [ ]:
!pip install -q transformers datasets scipy torch

## 2. Import Libraries

In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, Subset
from transformers import BertForSequenceClassification, BertTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
import matplotlib.pyplot as plt
from scipy import stats

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 3. Configuration

In [ ]:
# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Training configuration
MAX_LENGTH = 128  # Reduced for faster training in Colab
BATCH_SIZE = 8    # Reduced for Colab GPU memory
EPOCHS = 3
N_RUNS = 1        # Set to 1 for quick demo (use 3+ for robust results)
LEARNING_RATE = 2e-5
LOSS_THRESHOLD = 0.5

# For quick testing, use subset of data
USE_SUBSET = True
TRAIN_SUBSET_SIZE = 1000  # Use 1000 samples for quick demo
TEST_SUBSET_SIZE = 500

print(f"\nConfiguration:")
print(f"  Max Length: {MAX_LENGTH}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Runs: {N_RUNS}")
print(f"  Using Subset: {USE_SUBSET}")
if USE_SUBSET:
    print(f"  Train Subset: {TRAIN_SUBSET_SIZE}")
    print(f"  Test Subset: {TEST_SUBSET_SIZE}")

## 4. Dataset Class

In [ ]:
class IMDBDataset(Dataset):
    """Wrapper for HuggingFace IMDB dataset with tokenization."""

    def __init__(self, split='train', max_length=128, subset_size=None):
        print(f"Loading {split} dataset...")
        self.dataset = load_dataset('stanfordnlp/imdb', split=split)
        
        # Use subset if specified
        if subset_size is not None:
            indices = list(range(min(subset_size, len(self.dataset))))
            self.dataset = self.dataset.select(indices)
            print(f"Using subset of {len(self.dataset)} samples")
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item['text']
        label = item['label']

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

## 5. Batch Sampling Strategies

In [ ]:
def normal_batch_sampler(dataset, batch_size, **kwargs):
    """Standard random batch sampling."""
    n = len(dataset)
    indices = np.arange(n)
    np.random.shuffle(indices)
    for start in range(0, n, batch_size):
        yield indices[start:start + batch_size].tolist()


def loss_based_batch_sampler(dataset, batch_size, loss_history=None, threshold=LOSS_THRESHOLD, **kwargs):
    """
    Sample batches from examples with loss above threshold.
    First epoch: uses all examples.
    Subsequent epochs: only examples with loss > threshold.

    Always ensures at least batch_size examples are selected to prevent empty epochs.
    """
    if loss_history is None or loss_history.sum() == 0:
        # First epoch: use all examples
        indices = np.arange(len(dataset))
    else:
        # Filter to examples with loss above threshold
        indices = np.where(loss_history > threshold)[0]

        # IMPORTANT: Always ensure minimum number of samples to prevent empty epochs
        min_samples = batch_size
        if len(indices) < min_samples:
            # Not enough samples - pick top-k by loss instead
            top_k = min(min_samples, len(dataset))
            indices = np.argsort(loss_history)[-top_k:]  # Top-k highest loss samples
            print(f"  Filtering: Only {len(np.where(loss_history > threshold)[0])} examples above threshold, using top-{top_k} by loss instead")
        else:
            print(f"  Filtering: {len(indices)}/{len(dataset)} examples above threshold")

    np.random.shuffle(indices)

    # Yield batches, including any remaining samples in a final smaller batch
    for start in range(0, len(indices), batch_size):
        batch_indices = indices[start:start + batch_size]
        yield batch_indices.tolist()

## 6. Training Function

In [ ]:
def train_model(model, train_dataset, test_dataset, epochs, batch_size,
                batch_sampler_fn, strategy_name, seed=None):
    """Train BERT model with specified batch sampling strategy."""
    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    model = model.to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    # Calculate total training steps for scheduler
    n_samples = len(train_dataset)
    steps_per_epoch = n_samples // batch_size
    total_steps = steps_per_epoch * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # Track per-sample loss
    per_sample_loss = np.zeros(len(train_dataset))

    # Metrics storage
    train_accs, test_accs, train_losses, test_losses = [], [], [], []
    samples_per_epoch = []

    for epoch in range(epochs):
        model.train()
        correct, n_samples_epoch, running_loss = 0, 0, 0

        # Get batch iterator
        if 'loss_history' in batch_sampler_fn.__code__.co_varnames:
            batch_iter = batch_sampler_fn(train_dataset, batch_size, loss_history=per_sample_loss)
        else:
            batch_iter = batch_sampler_fn(train_dataset, batch_size)

        # Training loop
        for batch_indices in batch_iter:
            # Prepare batch
            batch = [train_dataset[i] for i in batch_indices]
            input_ids = torch.stack([item['input_ids'] for item in batch]).to(DEVICE)
            attention_mask = torch.stack([item['attention_mask'] for item in batch]).to(DEVICE)
            labels = torch.stack([item['label'] for item in batch]).to(DEVICE)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            # Compute loss per sample
            logits = outputs.logits
            loss_fn = nn.CrossEntropyLoss(reduction='none')
            losses_per_sample = loss_fn(logits, labels)
            loss = losses_per_sample.mean()

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            # Update metrics
            running_loss += loss.item() * len(batch_indices)
            predictions = logits.argmax(dim=1)
            correct += (predictions == labels).sum().item()
            n_samples_epoch += len(batch_indices)

            # Update per-sample loss history
            if 'loss_history' in batch_sampler_fn.__code__.co_varnames:
                for k, idx in enumerate(batch_indices):
                    per_sample_loss[idx] = losses_per_sample[k].item()

        # Epoch metrics
        train_acc = correct / n_samples_epoch if n_samples_epoch > 0 else 0
        train_loss = running_loss / n_samples_epoch if n_samples_epoch > 0 else 0
        train_accs.append(train_acc)
        train_losses.append(train_loss)
        samples_per_epoch.append(n_samples_epoch)

        # Evaluation
        test_acc, test_loss = evaluate(model, test_dataset, batch_size)
        test_accs.append(test_acc)
        test_losses.append(test_loss)

        print(f"[{strategy_name}] Epoch {epoch+1}/{epochs}: "
              f"train_acc={train_acc:.4f}, test_acc={test_acc:.4f}, "
              f"train_loss={train_loss:.4f}, samples={n_samples_epoch}")

    return {
        'train_acc': train_accs,
        'test_acc': test_accs,
        'train_loss': train_losses,
        'test_loss': test_losses,
        'samples_per_epoch': samples_per_epoch
    }

## 7. Evaluation Function

In [ ]:
def evaluate(model, dataset, batch_size):
    """Evaluate model on dataset."""
    model.eval()
    correct, total, total_loss = 0, 0, 0
    loss_fn = nn.CrossEntropyLoss()

    indices = list(range(len(dataset)))

    with torch.no_grad():
        for start in range(0, len(indices), batch_size):
            batch_indices = indices[start:start + batch_size]
            batch = [dataset[i] for i in batch_indices]

            input_ids = torch.stack([item['input_ids'] for item in batch]).to(DEVICE)
            attention_mask = torch.stack([item['attention_mask'] for item in batch]).to(DEVICE)
            labels = torch.stack([item['label'] for item in batch]).to(DEVICE)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            logits = outputs.logits
            loss = loss_fn(logits, labels)
            predictions = logits.argmax(dim=1)

            total_loss += loss.item() * len(batch_indices)
            correct += (predictions == labels).sum().item()
            total += len(batch_indices)

    return correct / total, total_loss / total

## 8. Load Datasets

In [ ]:
# Load datasets
train_dataset = IMDBDataset(
    split='train',
    max_length=MAX_LENGTH,
    subset_size=TRAIN_SUBSET_SIZE if USE_SUBSET else None
)

test_dataset = IMDBDataset(
    split='test',
    max_length=MAX_LENGTH,
    subset_size=TEST_SUBSET_SIZE if USE_SUBSET else None
)

print(f"\nDataset sizes:")
print(f"  Train: {len(train_dataset)}")
print(f"  Test: {len(test_dataset)}")

## 9. Run Experiments

### Strategy 1: Normal (Baseline)

In [ ]:
print("="*60)
print("STRATEGY 1: NORMAL (BASELINE)")
print("="*60)

# Create fresh model
model_normal = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

# Train
start_time = time.time()
results_normal = train_model(
    model_normal, train_dataset, test_dataset,
    EPOCHS, BATCH_SIZE,
    normal_batch_sampler, "Normal",
    seed=42
)
time_normal = time.time() - start_time

print(f"\nNormal strategy completed in {time_normal:.2f}s")
print(f"Final test accuracy: {results_normal['test_acc'][-1]:.4f}")

### Strategy 2: Loss-Based Batch Selection

In [ ]:
print("\n" + "="*60)
print("STRATEGY 2: LOSS-BASED BATCH SELECTION")
print("="*60)

# Create fresh model
model_loss_based = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

# Train
start_time = time.time()
results_loss_based = train_model(
    model_loss_based, train_dataset, test_dataset,
    EPOCHS, BATCH_SIZE,
    loss_based_batch_sampler, "Loss-Based",
    seed=42
)
time_loss_based = time.time() - start_time

print(f"\nLoss-based strategy completed in {time_loss_based:.2f}s")
print(f"Final test accuracy: {results_loss_based['test_acc'][-1]:.4f}")

## 10. Compare Results

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
epochs_range = np.arange(1, EPOCHS + 1)

# Test Accuracy
axes[0, 0].plot(epochs_range, results_normal['test_acc'], 'o-', label='Normal', linewidth=2)
axes[0, 0].plot(epochs_range, results_loss_based['test_acc'], 's-', label='Loss-Based', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Test Accuracy')
axes[0, 0].set_title('Test Accuracy Comparison')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Train Loss
axes[0, 1].plot(epochs_range, results_normal['train_loss'], 'o-', label='Normal', linewidth=2)
axes[0, 1].plot(epochs_range, results_loss_based['train_loss'], 's-', label='Loss-Based', linewidth=2)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Train Loss')
axes[0, 1].set_title('Train Loss Comparison')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Train Accuracy
axes[1, 0].plot(epochs_range, results_normal['train_acc'], 'o-', label='Normal', linewidth=2)
axes[1, 0].plot(epochs_range, results_loss_based['train_acc'], 's-', label='Loss-Based', linewidth=2)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Train Accuracy')
axes[1, 0].set_title('Train Accuracy Comparison')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Samples per Epoch
axes[1, 1].plot(epochs_range, results_normal['samples_per_epoch'], 'o-', label='Normal', linewidth=2)
axes[1, 1].plot(epochs_range, results_loss_based['samples_per_epoch'], 's-', label='Loss-Based', linewidth=2)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Number of Samples')
axes[1, 1].set_title('Samples per Epoch')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"\nNormal Strategy:")
print(f"  Final Test Accuracy: {results_normal['test_acc'][-1]:.4f}")
print(f"  Training Time: {time_normal:.2f}s")
print(f"  Samples per epoch: {results_normal['samples_per_epoch']}")

print(f"\nLoss-Based Strategy:")
print(f"  Final Test Accuracy: {results_loss_based['test_acc'][-1]:.4f}")
print(f"  Training Time: {time_loss_based:.2f}s")
print(f"  Samples per epoch: {results_loss_based['samples_per_epoch']}")

print(f"\nKey Observations:")
print(f"  - Loss-based filtering reduces samples in later epochs")
print(f"  - Potential speedup: {(time_normal/time_loss_based):.2f}x")
print(f"  - Accuracy difference: {(results_loss_based['test_acc'][-1] - results_normal['test_acc'][-1]):.4f}")

## 11. (Optional) Test on Sample Predictions

In [ ]:
# Test both models on a few examples
def predict_sentiment(model, text, tokenizer):
    model.eval()
    encoding = tokenizer(
        text,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        input_ids = encoding['input_ids'].to(DEVICE)
        attention_mask = encoding['attention_mask'].to(DEVICE)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = outputs.logits.argmax(dim=1).item()
    
    return "Positive" if prediction == 1 else "Negative"

# Test examples
test_texts = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "Terrible film. Complete waste of time and money.",
    "It was okay, nothing special but not bad either."
]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print("\nSample Predictions:")
print("="*60)
for i, text in enumerate(test_texts, 1):
    pred_normal = predict_sentiment(model_normal, text, tokenizer)
    pred_loss = predict_sentiment(model_loss_based, text, tokenizer)
    
    print(f"\nExample {i}:")
    print(f"  Text: {text}")
    print(f"  Normal Model: {pred_normal}")
    print(f"  Loss-Based Model: {pred_loss}")